# 2. CLEANSED LAYER

In [1]:
!pip install pandasql

"pip" no se reconoce como un comando interno o externo,
programa o archivo por lotes ejecutable.


In [2]:
import pandas as pd
import pandasql as psql

In [3]:
path_raw = 'C:\\Users\\anton\\OneDrive\\Documentos\\GitHub\\UniversityHack2024\\data\\01_Raw\\'
path_cleansed = 'C:\\Users\\anton\\OneDrive\\Documentos\\GitHub\\UniversityHack2024\\data\\02_Cleansed\\'

## 2.1 OF 123456.xlsx

In [5]:
# Ruta Excel para OF 123456
path_OF_01 = path_raw + 'OF 123456.xlsx'

In [6]:
# Guardamos el contenido en un DataFrame
df_OF_01 = pd.read_excel(path_OF_01)
print(df_OF_01.dtypes)

Orden                     int64
Número material           int64
Texto breve material     object
Lote                     object
Cantidad entregada      float64
Unidad de medida         object
dtype: object


In [5]:
# La consulta convierte el campo Lote a entero eliminando las barras ('/'), y cambiamos el nombre de las demás columnas.
consulta = """
    SELECT 
    CAST(REPLACE(`Lote`, '/', '') AS INTEGER) AS Lote, 
    Orden AS OF,
    `Cantidad entregada` AS Cantidad_entregada

    FROM df
    """
# Ejecutamos la consulta
df_OF_02 = psql.sqldf(consulta, locals())

In [93]:
# Guardar el resultado en un archivo Excel
df_OF_02.to_excel(f'{path_cleansed}OF_123456.xlsx', index=False)

# Mostrar el resultado
display(df_OF_02)

Lote                    int64
OF                      int64
Cantidad_entregada    float64
dtype: object
      Lote         OF  Cantidad_entregada
0    23019  200178572               13.80
1    23020  200179217               13.60
2    23021  200181620               13.50
3    23022  200182428               13.80
4    23023  200182429               13.70
..     ...        ...                 ...
202  24108  200205686               13.63
203  24106  200205695               13.69
204  24107  200205696               13.45
205  24109  200205687               11.35
206  24110  200205688               13.77

[207 rows x 3 columns]


## 2.2 Fases producción.xlsx

In [10]:
# Ruta Excel para Fases de producción
path_fases_01 = path_raw + 'Fases producción.xlsx'

### 2.2.1 PREINÓCULO

In [16]:
# Leer el archivo Excel y guardar el contenido en un DataFrame
df_preinoculo_01 = pd.read_excel(path_fases_01, sheet_name = "Preinóculo")
display(df_preinoculo_01.dtypes)

LOTE                               object
Fecha/hora inicio          datetime64[ns]
Fecha/hora fin             datetime64[ns]
ph_línea 1                        float64
ph_línea 2                        float64
ph_línea 3                         object
turbidez_línea 1                  float64
turbidez_línea 2                  float64
turbidez_línea 3                   object
linea_utilizada_línea 1             int64
linea_utilizada_línea 2             int64
linea_utilizada_línea 3             int64
dtype: object

In [19]:
# Convertimos la columnas 'ph_línea 3' y 'turbidez_línea 3' a numérico y forzamos los valores no numéricos a NaN
df_preinoculo_01['ph_línea 3'] = pd.to_numeric(df_preinoculo_01['ph_línea 3'], errors='coerce')
df_preinoculo_01['turbidez_línea 3'] = pd.to_numeric(df_preinoculo_01['turbidez_línea 3'], errors='coerce')
print(df_preinoculo_01.dtypes)

LOTE                               object
Fecha/hora inicio          datetime64[ns]
Fecha/hora fin             datetime64[ns]
ph_línea 1                        float64
ph_línea 2                        float64
ph_línea 3                        float64
turbidez_línea 1                  float64
turbidez_línea 2                  float64
turbidez_línea 3                  float64
linea_utilizada_línea 1             int64
linea_utilizada_línea 2             int64
linea_utilizada_línea 3             int64
dtype: object


In [ ]:
df_preinoculo_01.to_excel(f'{path_cleansed}Fases producción_Preinóculo.xlsx', index=False)

### 2.2.2 INÓCULO

In [21]:
# Leer el archivo Excel y guardar el contenido en un DataFrame
df_inoculo_01 = pd.read_excel(path_fases_01, sheet_name = "Inóculo")
display(df_inoculo_01.dtypes)

LOTE                                object
ID bioreactor                        int64
Fecha/hora inicio           datetime64[ns]
Fecha/hora fin              datetime64[ns]
Volumen de cultivo                 float64
Turbidez inicio cultivo            float64
Turbidez final culttivo            float64
Viabilidad final cultivo            object
dtype: object

In [ ]:
df_inoculo_01.to_excel(f'{path_cleansed}Fases producción_Inóculo.xlsx', index=False)

### 2.2.3 CULTIVO FINAL

In [ ]:
# Leemos el archivo Excel y guardamos el contenido en un DataFrame
df_cultivo_01 = pd.read_excel(path_fases_01, sheet_name = "Cultivo final")

In [ ]:
# La consulta convierte los campos Lote, Turbidez y Glucosa g/L a valores numéricos, manejando excepciones como NULL.
consulta = """
    SELECT 
        CAST(Lote AS Integer) AS Lote,
        Fecha,
        CAST(NULLIF(Turbidez, '-') AS Float) AS Turbidez,
        Viabilidad,
        CASE
            WHEN `Glucosa g/L` REGEXP '^[0-9]+\\.[0-9]+$' THEN CAST(`Glucosa g/L` AS Float)
            ELSE NULL
        END AS Glucosa
    FROM df_cultivoIPC_01
"""
df_cultivo_02 = psql.sqldf(consulta, locals())

# Cambiamos, con la librería pandas, el tipo de campo 'Fecha' a DATETIME porque la librería pandasql no tiene la función
df_cultivo_02['Fecha'] = pd.to_datetime(df_cultivo_02['Fecha'])

In [ ]:
# Guardamos el resultado en un archivo Excel
df_cultivo_02.to_excel(f'{path_cleansed}Fases producción_Cultivo final.xlsx', index=False)

## 2.3 Fases producción_test.xlsx

In [23]:
# Ruta Excel para Fases de producción_test
path_cultivotest_01 = path_raw + 'Fases producción_test.xlsx'

In [24]:
# Leer el archivo Excel y guardar el contenido en un DataFrame
df_cultivotest_01 = pd.read_excel(path_cultivotest_01)
# Eliminar la columna 'Producto 2'
df_cultivotest_02 = df_cultivotest_01.drop(columns=['Producto 2'])

In [ ]:
df_cultivotest_02.to_excel(f'{path_cleansed}Fases producción_test_Cultivo final.xlsx', index=False)

## 2.4 Cinéticos IPC.xlsx

In [26]:
# Ruta Excel para Cinéticos IPC
path_IPC_01 = path_raw + 'Cinéticos IPC.xlsx'

### 2.4.1 INÓCULO

In [28]:
df_inoculoIPC_01 = pd.read_excel(path_IPC_01, sheet_name = 'Inóculos')
print(df_inoculoIPC_01.dtypes)

Lote                  object
Fecha         datetime64[ns]
Turbidez             float64
Viabilidad           float64
dtype: object


In [ ]:
df_inoculoIPC_01.to_excel(f'{path_cleansed}CineticosIPC_Inoculos.xlsx', index=False)

### 2.4.2 CULTIVO FINAL

In [ ]:
# Leemos el archivo Excel y guardamos el contenido en un DataFrame
df_cultivoIPC_01 = pd.read_excel(path_IPC_01, sheet_name='Cultivos finales')

In [ ]:
# La consulta convierte los campos Lote, Turbidez y Glucosa g/L a valores numéricos, manejando excepciones como NULL.
consulta = """
    SELECT 
        CAST(Lote AS Integer) AS Lote,
        Fecha,
        CAST(NULLIF(Turbidez, '-') AS Float) AS Turbidez,
        Viabilidad,
        CASE
            WHEN `Glucosa g/L` REGEXP '^[0-9]+\\.[0-9]+$' THEN CAST(`Glucosa g/L` AS Float)
            ELSE NULL
        END AS Glucosa
    FROM df_cultivoIPC_01
"""
df_cultivoIPC_02 = psql.sqldf(consulta, locals())

# Cambiamos, con la librería pandas, el tipo de campo 'Fecha' a DATETIME porque la librería pandasql no tiene la función
df_cultivoIPC_02['Fecha'] = pd.to_datetime(df_cultivoIPC_02['Fecha'])

Lote                   object
Fecha          datetime64[ns]
Turbidez               object
Viabilidad            float64
Glucosa g/L            object
dtype: object


In [ ]:
# Guardamos el resultado en un archivo Excel
df_cultivoIPC_02.to_excel(f'{path_cleansed}CineticosIPC_Cultivos.xlsx', index=False)

# Mostrar el resultado
display(df_cultivoIPC_02)

Lote                   int64
Fecha         datetime64[ns]
Turbidez             float64
Viabilidad           float64
Glucosa              float64
dtype: object
       Lote               Fecha  Turbidez    Viabilidad  Glucosa
0     23019 2023-03-21 07:30:00     17.28  8.333333e+06    1.744
1     23019 2023-03-21 11:00:00     17.28  1.397333e+07    1.968
2     23019 2023-03-21 14:00:00     18.40  1.688000e+07    1.920
3     23019 2023-03-21 18:00:00     19.76  2.474667e+07    1.880
4     23019 2023-03-22 07:00:00     39.12  1.000000e+08    1.568
...     ...                 ...       ...           ...      ...
1906  24110 2024-07-04 15:30:00     58.40  8.053333e+07    1.584
1907  24110 2024-07-04 19:30:00     69.76  1.021333e+08    1.248
1908  24110 2024-07-05 06:30:00    100.80  1.525333e+08    0.600
1909  24110 2024-07-05 08:00:00    104.80  1.514667e+08    0.520
1910  24110 2024-07-05 10:10:00    104.80  1.573333e+08    0.592

[1911 rows x 5 columns]


### 2.4.3 CENTRIFUGACIÓN

In [31]:
df_centIPC_01 = pd.read_excel(path_IPC_01, sheet_name = 'Centrifugación')
print(df_centIPC_01.dtypes)

Lote                        object
Centrífuga                   int64
Centrifugada (1 o 2)         int64
Volumen centrifugado (L)     int64
Turbidez                    object
dtype: object


In [ ]:
df_centIPC_01.to_excel(f'{path_cleansed}CineticosIPC_Centrifugacion.xlsx', index=False)

## 2.5 Horas inicio fin centrífugas.xlsx

In [32]:
path_horascentrifuga_01 = path_raw + 'Horas inicio fin centrífugas.xlsx'

In [34]:
df_horascentrifuga_01 = pd.read_excel(path_horascentrifuga_01)
print(df_horascentrifuga_01.dtypes)

EQUIPO        int64
Operación    object
Orden         int64
DATEVALUE    object
dtype: object


In [ ]:
df_horascentrifuga_01.to_excel(f'{path_cleansed}Centrifugacion.xlsx')

## 2.6 Biorreactores

### 2.6.1 Grandes (13169, 13170, 14614, 14615, 14616, 14617)

In [5]:
# Juntamos, en primer lugar, los Excel de los diferentes biorreactores grandes en un sólo Excel
# A continuación, añadimos una columna con el ID de cada biorreactor
path_biograndes_01 = path_raw + 'biorreactores_grandes.xlsx'

df_biograndes_01 = pd.read_excel(path_biograndes_01)

FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\anton\\OneDrive\\Documentos\\GitHub\\UniversityHack2024\\data\\01_Raw\\biorreactores_grandes.xlsx'